### Building a model to predict symptomatic status of LQTS using clinical, genetic and demographic features                
#### *Written by Peng Xu (xu.peng@mayo.edu)*    

#### Dataset
##### The whole cohort has LQTS patients and non-LQTS patients including QTc, Age, Gender and LQTs, family history features, etc.

#### Aim:
##### To predict the symptomatic status of LQTS using all of possible avaible features so the new model can outperform the Prior model.....

## 1. Load the libraries

In [1]:
# import the nessacery modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import math 

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# # Function for splitting training and test set
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel


# Function for creating model pipelines
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# For standardization
from sklearn.preprocessing import StandardScaler

# Helper for cross-validation
from sklearn.model_selection import GridSearchCV

# Classification metrics (added later)
from sklearn.metrics import roc_curve, auc, roc_auc_score, recall_score
from sklearn.metrics import accuracy_score

## 2. Load the whole cohort dataset and select LQTS dataset

In [2]:
def data_loader(file_name):
    """Load the dataset and print the row# and column#."""
    df = pd.read_csv(file_name)
    display(df.head(3))
    print(f"{df.shape[0]} samples and {df.shape[1]} columns.")
    return df

def columns_list(df):
    print("Index: Column Name")
    for i in range(0,len(df.columns)):
        print(str(i),":", df.columns[i])

fileName = 'LQTS_Proband_09062019.csv'
df_raw = data_loader(fileName)

## 3. EDA

In [ ]:
# functions
plt.style.use('_classic_test')

def distplot(obs1, obs2, bins, x1, x2, figName, style):
    """Plot the observation distribution"""
    
    sns.set(style='white')
    
    if style == 'hist':
        plt.ylabel('The number of patients')
        sns.distplot(obs1,bins=bins, color='red', kde=False)
        if obs2 is not None:
            sns.distplot(obs2, bins =bins, color='black', kde=False)
        plt.ylabel('The number of patients')
    
    elif style =='curve':
        sns.distplot(obs1,bins=bins, color='red', hist=False, kde_kws={"shade": True})
        if obs2 is not None:
            sns.distplot(obs2, bins =bins, color='black', hist=False, kde_kws={"shade": True})
    elif style =='hist+curve':
        sns.distplot(obs1,bins=bins, color='red')
        if obs2 is not None:
            sns.distplot(obs2, bins =bins, color='black')
        
    plt.xlim(x1, x2)
    plt.savefig(figName, dpi=300)
    plt.show()
    
def cat_feature(df,feature_col, label_col, label_value):
    """Statistic of feature agaist label"""
    i = label_value
    for f in df[feature_col].unique():
        penc = df[(df[feature_col] == 
                   f) & (df[label_col] == i)].shape[0] / df[df[feature_col] == f].shape[0]
        print(f'{f} in {feature_col}:  {100*round(penc,2)}%')
        
df_LQTS = df_raw
df_symp = df_LQTS[df_LQTS['SymptomLifeTime'] == 1]
df_LQTS.describe()        

#### Feature1 : Proband
###### 70.1% Proband family has the symptoms while only 16.5% related family members are symptomatic
#### Feature 2: Age at Event
##### Age at the first cardio event will be split before 30 and after 30 because more symptomatic patients are present before 30.
#### Feature 3: Gender at Event
##### 40.2% female has the symptoms while only 27.4% male are symptomatic.
#### Feature 4: Genetic variant
##### LQT1,2,3 and Multiple Mutation
#### Feature 5: Family history of sudden cardiac death 
#### Is there a family history of sudden cardiac death under the age of 45?  If Yes, there is a family history. For sympmatic LQTS patients, 
##### No :  40.17% ; Yes :  29.87% ;Unknown:  50.0%
#### Feature 6: History of LQTS?
##### Is there a family history of LQTS?  If there is a LQTS listed, then they have a family history. For sympmatic LQTS patients, 
##### Yes :  29%
#### Feature 7: QTc
##### QTc comes from computed QTc value at baseline for the patient (typically the first visit with Dr. Ackerman) with Dr. Ackerman correction.

In [ ]:
######################################### Features  ################################
#############Demographic
## Age
obs1 = df_LQTS[df_LQTS['SymptomLifeTime'] == 1]['age_first_cardia_event']
obs2 = df_LQTS[df_LQTS['SymptomLifeTime'] == 0]['age_first_cardia_event']
x1 = 0
x2 = 80
style = 'hist+curve'
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 65, 75]
figName = 'Age at LQTS whole cohort.png'
distplot(obs1, obs2, bins, x1, x2, figName, style)
##Gender
df_LQTS.groupby(['Gender','SymptomLifeTime'])['MRN#'].count()
print(f'{round(100*305/(305+453),1)}% female has the symptoms.')
print(f'{round(100*151/(151+400),1)}% male has the symptoms.')

#############Genetic
print('Symptom in Life(yes:1, no:0)')
print('---------------------------')
for col in ['LQTS 1 (KCNQ1)','LQTS 2 (KCNH2)','LQTS 3 (SCN5A)','LQTS Multiple Mutations','LQTS Minor Genes']:
    cat_feature(df_LQTS,col,'SymptomLifeTime',1)
    print() 

#f, ax = plt.subplots(figsize=(10, 10))
data = round(df_LQTS['LQTS 1 (KCNQ1)','LQTS 2 (KCNH2)','LQTS 3 (SCN5A)','LQTS Multiple Mutations','LQTS Minor Genes'].corr(), 2)
sns.set_style("whitegrid")
sns.heatmap(data=round(data, annot = True, square = True)
plt.show()


#############Clinical
##Proband
print('Category                % with symptoms')
df_LQTS.groupby(['Proband','SymptomLifeTime'])['MRN#'].count()
print(f'{round(100*314/(313+135),1)}% Proband family has the symptoms')
print(f'{round(100*142/(142+718),1)}% Related family member has the symptoms')
##Family history of sudden cardiac death
cat_feature(df_LQTS,'History of SCD','SymptomLifeTime', 1)
##Family of LQTC
cat_feature(df_LQTS,'History of LQTS','SymptomLifeTime', 1)
## QTc
obs1 = df_LQTS[df_LQTS['SymptomLifeTime'] == 1]['QTc']
obs2 = df_LQTS[df_LQTS['SymptomLifeTime'] == 0]['QTc']
x1 = 350
x2 = 751
style = 'hist+curve'
bins_QTc = [370, 390, 410, 430, 450, 470, 490, 510, 530, 550, 570, 590, 610,630,751]
figName = 'QTc_all_features.png'
distplot(obs1, obs2, bins_QTc, x1, x2, figName, style)

## 4. Feature engineering 

#### Based on the feature selection results, we will use 
1. clinical variables: QTc, Gender, Age, Family history of Sudden Cardio Death, LQTS family
1. genetic variables:  LQTS 1 (KCNQ1),LQTS 2 (KCNH2),LQTS 3 (SCN5A),LQTS Multiple Mutation,LQTS Minor Genes.

In [ ]:
########################## PreProcessing ################################
#######1. Transform Proband, Gender as dummy variable            ########
#######2. Transform History of SCD as dummy variable             ########
#######3. Transform History of LQTS as dummy variable            ########
#######4. Transform Age(<=30 or >30) as dummy variable           ########
#######5. Add QTc as dummary variable (>=500 or <500ms)          ########
#######6. Standerscale the QTc                                   ########

clinical_cols = ['Proband','QTc','Gender','age_first_cardia_event',
                 'History of SCD','History of LQTS']
genetic_cols = ['LQTS 1 (KCNQ1)','LQTS 2 (KCNH2)','LQTS 3 (SCN5A)',
                'LQTS Multiple Mutations','LQTS Minor Genes']
target_var = ['SymptomLifeTime']
feature_cols = clinical_cols+genetic_cols+target_var
df_features = df_LQTS[feature_cols]

def LQTS_PreProcessing(df):
    """PreProcessing the features for LQTS"""
    replace_val = {'Proband':{'Proband':1, 'Related family member':0},
                   'Gender': {'Female': 1, 'Male':0},
                   'History of SCD':{'Yes':1, 'No':0, 'Unknown':0}, 
                   'History of LQTS':{'LQTS':1, 'None':0, 'Unknown':0, '0':0,
                                      'ARVC':0,'Other':0}}
    df_prep = df.replace(replace_val)
    df_prep['Age(<=30)']=[1 if age <=30 else 0 for age in df['age_first_cardia_event']]
    df_prep['QTc(>=500)']=[1 if QTc >=500 else 0 for QTc in df['QTc']]
    #QTc standard scaler
    QTc_mean = df['QTc'].mean()
    QTc_std = df['QTc'].std()
    df_prep['QTc_standard']=[(age-QTc_mean)/QTc_std for age in df['QTc']]
    
    return df_prep

df_preprocess = LQTS_PreProcessing(df_features)

#### Save the file as feature variables
feature_cols = ['Proband','QTc_standard','QTc(>=500)','Gender','Age(<=30)',
                'History of SCD','History of LQTS',
                'LQTS 1 (KCNQ1)','LQTS 2 (KCNH2)','LQTS 3 (SCN5A)',
                'LQTS Multiple Mutations', 'LQTS Minor Genes']
target_var = ['SymptomLifeTime']

df_model = df_preprocess[feature_cols + target_var]
df_model.to_csv('LQTS_model_whole_cohort_proband.csv') # save the file for further maching learning

## Read the Proprecessing
#df_model = pd.read_csv('LQTS_model_whole_cohort_proband.csv')

In [ ]:
########################## Modeling dataset #############################
#######1. Proband                                                ########
#######2. No Proband                                             ########
#######3. Priori                                                 ########

df_ml_no_proband = df_model[['QTc_standard', 'Gender', 'Age(<=30)', 
                             'History of SCD','History of LQTS', 
                             'LQTS 1 (KCNQ1)', 'LQTS 2 (KCNH2)', 'LQTS 3 (SCN5A)',
                             'LQTS Multiple Mutations', 'LQTS Minor Genes', 
                             'SymptomLifeTime']]

df_ml_proband = df_model[['Proband','QTc_standard', 'Gender', 'Age(<=30)', 
                          'History of SCD','History of LQTS', 
                          'LQTS 1 (KCNQ1)', 'LQTS 2 (KCNH2)', 'LQTS 3 (SCN5A)',
                          'LQTS Multiple Mutations', 'LQTS Minor Genes', 
                          'SymptomLifeTime']]

df_ml_Priori = df_model[['LQTS 1 (KCNQ1)', 
                         'LQTS 2 (KCNH2)',
                         'LQTS 3 (SCN5A)',
                         'QTc(>=500)', 
                         'Gender','SymptomLifeTime',]]

## 5. Machine learning

#### Run the machine learning pipeline and find the winner model

In [ ]:
def getRocData(hot_y,y_score,wantedClass=0):
    """Use the The Closest to (0,1) as threshold"""
    i =wantedClass
    if ((hot_y.shape[1:]==()) & (y_score.shape[1:]==()) ): # Vector ROC
        hot_y_use = hot_y 
        y_score_use = y_score 
    else:
        hot_y_use = hot_y[:, i]
        y_score_use = y_score[:, i]
    
    fpr , tpr , th  = roc_curve(hot_y_use, y_score_use)
    roc_auc  = auc(fpr , tpr )
    dist=np.sqrt((1-tpr )**2+(fpr)**2)
    optimalIndex = np.argmin(dist)
    return fpr , tpr,th, roc_auc, optimalIndex

In [ ]:
models = {
    'Lasso Regression':LogisticRegression(penalty='l1',solver='liblinear',random_state=42),
    'Gaussian NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    #'SVM': SVC(probability=True, random_state=42,verbose=False),
    'Linear SVM': SVC(kernel='linear',probability=True,random_state=42,verbose=False)
    }
#hyperparameters
l1_hyperparameters = {'C': [1e-5,1e-4,1e-3,1e-2,1e-1,1,10,1e2,1e3,1e4,1e5]}
ngb_hyperparameters = {}
rf_hyperparameters = {'n_estimators': [50, 100, 200]}
                      #'max_features': ['sqrt', 0.33]}
svm_linear_hyperparameters = {'C': [0.01,0.1,0.5,1.0,5.0,10,20,40]}
# Create hyperparameters dictionary
hyperparameters = {'Lasso Regression': l1_hyperparameters,
                   'Gaussian NaiveBayes': ngb_hyperparameters,
                   'RandomForest' : rf_hyperparameters, 
                   #'SVM': svm_hyperparameters,
                   'Linear SVM':svm_linear_hyperparameters}

def best_estimator(X, y, kfold, scoring, model, hyperparameter):
    """Classifier with Logistic Regression, Random Forest, SVM, Naive Bayes
    """
    # Train the model and return the best estimator using GridSeearCV
    clf = GridSearchCV(model,
                       hyperparameter,
                       scoring=scoring,
                       cv=kfold,
                       n_jobs=-1)
    clf.fit(X, y)
    return clf.best_estimator_

def data_split(df, label, random_seed):
    """Split the data into training and testing set"""
    # Split the data
    y = df[label]
    X = df.drop(label, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=random_seed,
                                                        stratify=df[label])
    return X_train, X_test, y_train, y_test 

def model_performance_test(model, X_test, y_test):
    
    clf = model
    #testing dataset to get the predict and proba
    y_proba = clf.predict_proba(X_test)
    
    #Optimal threthod
    fpr, tpr, th, roc_auc, optimalIndex = getRocData(y_test.values,y_proba[:,1])
    optimalTh = th[optimalIndex]
    est_y_binary = y_proba[:,1]>=optimalTh
    tn, fp, fn, tp = confusion_matrix(y_test.values, est_y_binary).ravel()
    
    #print(optimalTh)
    auc_score = round(roc_auc, 3)
    accuracy = round((tn+tp)/(tp+tn+fp+fn),3)
    sensitivity = round(tp/(tp+fn),3)
    specificity = round(tn/(tn+fp),3)
    
    metric_score = [auc_score, accuracy, sensitivity, specificity]
    #cm = confusion_matrix(y_test.values, est_y_binary)
    return metric_score
    
    
def machine_learning_pipeline(df,label, kfold, scoring):
    # split the data
    X_train, X_test, y_train, y_test = data_split(df, label)
    
    # find the best estimators and their performace metrics
    model_metrics = {}
    model_metrics['Metrics'] = ['ROC AUC Score','Accuracy','Sensitivity','Specificity']
    for model_name in models.keys():
        print(model_name)
        clf = best_estimator(X_train, y_train, kfold, scoring, 
                             models[model_name], 
                             hyperparameters[model_name])
        clf.fit(X_train, y_train)
        metric_score, cm = model_performance_test(clf, X_test, y_test)
        model_metrics[model_name] = metric_score
        
    df = pd.DataFrame.from_dict(model_metrics)
    df.set_index('Metrics', inplace=True)
    
    df_sorted_by_auc = df.sort_values(by='ROC AUC Score', axis=1, ascending=False)
    display(df_sorted_by_auc)
    
def boostrap(df):
    """Returen the boostrapping model performance results"""
    for name in ['Lasso Regression','Gaussian NaiveBayes','RandomForest','Linear SVM']:
        auc_list = []
        acc_list = []
        sen_list = []
        spe_list = []
        for random_seed in range(0,1000):
            X_train, X_test, y_train, y_test = data_split(df,'SymptomLifeTime', random_seed)
            clf = best_estimator(X_train, y_train, 5, 'roc_auc', models[name], hyperparameters[name])
            clf.fit(X_train, y_train)
            metric_score = model_performance_test(clf, X_test, y_test)
            auc_score, accuracy, sensitivity, specificity = metric_score
            auc_list.append(auc_score)
            acc_list.append(accuracy)
            sen_list.append(sensitivity)
            spe_list.append(specificity)
        print(name)
        print(f'AUC: {np.percentile(auc_list,50)},{np.percentile(auc_list,2.5)},{np.percentile(auc_list,97.5)}')
        print(f'Accuracy: {np.percentile(acc_list,50)},{np.percentile(acc_list,2.5)},{np.percentile(acc_list,97.5)}')
        print(f'Sensitivity: {np.percentile(sen_list,50)},{np.percentile(sen_list,2.5)},{np.percentile(sen_list,97.5)})')
        print(f'Specificity: {np.percentile(spe_list,50)},{np.percentile(spe_list,2.5)},{np.percentile(spe_list,97.5)})')

In [ ]:
### Calculate the 
boostrap(df_proband)
boostrap(df_no_Proband)
boostrap(df_Priori)

### Figure 1

In [ ]:
## Plot the model
def roc_matrix(model_name, df, label):
    X_train, X_test, y_train, y_test = data_split(df, label)
    model = best_estimator(X_train, y_train, 5, 'roc_auc',models[model_name], hyperparameters[model_name])
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)
    pred = [p[1] for p in pred]
    fpr, trp, _ = roc_curve(y_test, pred)
    auc_score   = auc(fpr, trp)
    return fpr, trp, round(auc_score,3);

# Initialize figure
fig = plt.figure(figsize=(8,8))
plt.title('ROC(Receiver Operating Characteristic)')
# Plot ROC curve


metric = roc_matrix('Lasso Regression', df_ml_use, 'SymptomLifeTime')
frp, trp, auc_score = metric[0], metric[1], metric[2]
plt.plot(frp,trp,label=f'AUC = {auc_score}')

plt.legend(loc='lower right')
# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')
# Axes limits and labels
# plt.xlim([0,1])
# plt.ylim([0,1])
plt.ylabel('Sensitivity',fontsize=12)
plt.xlabel('1 - Specificity',fontsize=12)


plt.savefig('ROC_all_features_Proband.png', dpi=300)

### Figure 2

In [ ]:
## Plot the model
def roc_matrix(model_name, df, label):
    X_train, X_test, y_train, y_test = data_split(df, label)
    model = best_estimator(X_train, y_train, 5, 'roc_auc',models[model_name], hyperparameters[model_name])
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)
    pred = [p[1] for p in pred]
    fpr, trp, _ = roc_curve(y_test, pred)
    auc_score   = auc(fpr, trp)
    return fpr, trp, round(auc_score,3);

# Initialize figure
fig = plt.figure(figsize=(8,8))
plt.title('ROC(Receiver Operating Characteristic)')
# Plot ROC curve
metric = roc_matrix('Lasso Regression', df_ml_use, 'SymptomLifeTime')
frp, trp, auc_score = metric[0], metric[1], metric[2]
plt.plot(frp,trp,label=f"Mayo's Features(AUC = {auc_score})")

metric = roc_matrix('Lasso Regression', df_ml_Priori, 'SymptomLifeTime')
frp, trp, auc_score = metric[0], metric[1], metric[2]
plt.plot(frp,trp,label=f"Priori's Features(AUC = {auc_score})")

plt.legend(loc='lower right')
# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')
# Axes limits and labels
# plt.xlim([0,1])
# plt.ylim([0,1])
plt.ylabel('Sensitivity',fontsize=12)
plt.xlabel('1 - Specificity',fontsize=12)


plt.savefig('ROC_Mayo_Priori.png', dpi=300)

## 5. Input Contribution
#### Return the coefficients to determine the input contribution.

In [ ]:
def model_input_contriubution(df):
    """Return the Logistic Regression coefficients"""
    X_train, X_test, y_train, y_test = data_split(df, 'SymptomLifeTime')
    model = best_estimator(X_train, y_train, 5, 'roc_auc',models['Lasso Regression'], hyperparameters['Lasso Regression'])
    model.fit(X_train, y_train)
    print(f"Input name: {X_train.columns}")
    print(f"Model Coefficience: {model.coef_}")

In [ ]:
### Proband
model_input_contriubution(df_model_proband)
### no proband
model_input_contriubution(df_model_no_proband)
### 
model_input_contriubution(df_Priori)

## 6. Save model

In [ ]:
# Save winning model as final_model.pkl
X_train, X_test, y_train, y_test = data_split(df_ml_proband, 'SymptomLifeTime')
model = best_estimator(X_train, y_train, 5, 'roc_auc',models['Lasso Regression'], hyperparameters['Lasso Regression'])
model.fit(X_train, y_train)

import pickle
with open('LQTS_risk_model.pkl', 'wb') as f:
    pickle.dump(model, f)